In [1]:
%defaultDatasource jdbc:h2:../data/interim/disksql

In [2]:
DROP TABLE IF EXISTS Mortalidade;


CREATE TABLE Mortalidade(
    Ano INTEGER,
    Estado VARCHAR(20),
    CodigoEstado INTEGER,
    Mortes INTEGER,
    População INTEGER,
    Mortes100K DOUBLE,
    PRIMARY KEY(Estado,Ano)
) AS SELECT
    Year,State,StateCode,Deaths,Population,CrudeRate
FROM CSVREAD('../data/processed/lungcancer.csv');

## Todas as taxas de mortalidade a seguir se referem a mortes por câncer de pulmão

### Top 5 mortalidade por 100 mil habitantes

In [3]:
SELECT *
FROM Mortalidade
ORDER BY Mortalidade.Mortes100k DESC
LIMIT 5

### Top 5 estado com maior mortalidade média por 100 mil habitantes de 1968 a 2018

In [4]:
SELECT Estado,AVG(Mortes100k) avg_mortes100k
FROM Mortalidade
GROUP BY Estado
ORDER BY avg_mortes100k DESC
LIMIT 5

### Top 10 anos com o maior número de mortes (valores absolutos)

In [5]:
SELECT Ano,SUM(mortes) TotalDeMortes
FROM Mortalidade
GROUP BY Ano
ORDER BY TotalDeMortes DESC
LIMIT 10

### Mortalidade nacional por 100 mil habitantes

In [6]:
SELECT Ano,Mtotal,Ptotal,CAST (Mtotal AS DECIMAL)/Ptotal*100000 Mortes100k
FROM(
    
    SELECT Ano,SUM(Mortes) Mtotal,SUM(População) Ptotal
    FROM Mortalidade
    GROUP BY Ano
    
)
ORDER BY Ano ASC

### Estados cuja média da mortalidade por 100 mil habitantes de 1968 a 2018 supera a média da mortalidade nacional de 1968 a 2018

In [7]:
SELECT M.Estado

FROM
    (SELECT Estado,AVG(Mortes100k) avg_mortes100k
    FROM Mortalidade
    GROUP BY Estado) as M
WHERE
    M.avg_mortes100k > (SELECT AVG(CAST (Mtotal AS DECIMAL)/Ptotal*100000)
                        FROM(
                            SELECT Ano,SUM(Mortes) Mtotal,SUM(População) Ptotal
                            FROM Mortalidade
                            GROUP BY Ano)
                        )

### Estados que superaram a mortalidade nacional por 100 mil habitantes por ano

In [8]:
SELECT Mortalidade.Ano, Mortalidade.Estado, Mortalidade.mortes100k,Nacional.mortes100kNacional
FROM 
    Mortalidade, 
    (SELECT Ano,Mtotal,Ptotal,CAST (Mtotal AS DECIMAL)/Ptotal*100000 mortes100kNacional
    FROM(

        SELECT Ano,SUM(Mortes) Mtotal,SUM(População) Ptotal
        FROM Mortalidade
        GROUP BY Ano

    )) AS Nacional
WHERE Mortalidade.Ano = Nacional.Ano and Mortalidade.mortes100k > Nacional.mortes100kNacional
ORDER BY Ano ASC,Mortes100k DESC